In [2]:
##%pip install ipynb

In [3]:
from ipynb.fs.full.twscrape import *
import pandas as pd

In [4]:
user_name = input("Input the username: ")
limit = int(input("Input the amount of tweets to get: "))
result = await gettweets(user_name, limit)
##print(result)

2023-11-09 12:17:54.335 | WARNING  | twscrape.accounts_pool:add_account:76 - Account BingBong2397074 already exists
2023-11-09 12:17:54.358 | WARNING  | twscrape.accounts_pool:add_account:76 - Account BBong20237297 already exists
2023-11-09 12:17:54.442 | WARNING  | twscrape.accounts_pool:relogin:164 - No usernames provided


user login: User(id=25073877, id_str='25073877', url='https://twitter.com/realDonaldTrump', username='realDonaldTrump', displayname='Donald J. Trump', rawDescription='45th President of the United States of America🇺🇸', created=datetime.datetime(2009, 3, 18, 13, 46, 38, tzinfo=datetime.timezone.utc), followersCount=87490379, friendsCount=51, statusesCount=59120, favouritesCount=0, listedCount=112952, mediaCount=4967, location='', profileImageUrl='https://pbs.twimg.com/profile_images/874276197357596672/kUuht00m_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/25073877/1604214583', protected=None, verified=False, blue=True, blueType=None, descriptionLinks=[TextLink(url='http://DonaldJTrump.com', text='DonaldJTrump.com', tcourl='https://t.co/mjKdTvQzae')], _type='snscrape.modules.twitter.User') 

user id: 25073877 

[Tweet(id=1694886846050771321, id_str='1694886846050771321', url='https://twitter.com/realDonaldTrump/status/1694886846050771321', date=datetime.datetime(202

In [5]:
##type(result)

In [6]:
df_tweets = pd.DataFrame(result)

In [7]:
##df_tweets.head()

In [8]:
##for column in df_tweets.columns:
##    print(column, " ", type(column))

In [9]:
## avoid sql keywords
df_tweets.rename(columns={'url': 'tweet_url', 'date': 'publish_date','user': 'tweet_user'}, inplace=True)

In [10]:
##df_tweets.head()

In [11]:
##%pip install psycopg2

In [12]:
import psycopg2

In [13]:
conn = psycopg2.connect(dbname="postgres",user="postgres", password="postgres", port="5432", host="localhost")
conn.autocommit = True
cursor = conn.cursor()

In [14]:
import sys
import traceback

In [15]:
## selection of usernames from mentionedUsers

##i = 0
##for index, row in df_tweets.iterrows():
##    i=i+1
##    (print(list(pd.DataFrame(row.mentionedUsers)['username'])) if str(row.mentionedUsers) != '[]' else print(str(row.mentionedUsers)))
##    if i == 16:
##        break    

In [16]:
i = 0
for index, row in df_tweets.iterrows():
    i=i+1
    ##(print(list(pd.DataFrame(row.mentionedUsers)['username'])) if str(row.mentionedUsers) != '[]' else print(str(row.mentionedUsers)))
    print(row.retweetedTweet['rawContent']) if (row.retweetedTweet != None) else print(row.retweetedTweet)
    if i == 10:
        break    

None
None
None
None
None
None
None
None
None
None


In [25]:
df_tweets['inReplyToTweetId'].unique()

array([           nan, 1.34596698e+18, 1.34579820e+18])

In [18]:
for index, row in df_tweets.iterrows():
    try:
        cursor.execute('''INSERT INTO Tweets (
                            id,
                            id_str,
                            tweet_url,
                            publish_date,
                            tweet_user,
                            lang,
                            rawcontent,
                            replycount,
                            retweetcount,
                            likecount,
                            quotecount,
                            conversationid,
                            hashtags,
                            cashtags,
                            mentionedusers,
                            links,
                            viewcount,
                            retweetedtweet,
                            quotedtweet,
                            place,
                            coordinates,
                            inreplytotweetid,
                            inreplytouser,
                            source,
                            sourceurl,
                            sourcelabel,
                            media,
                            _type
                        )
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);''', 
                       (row.id,
                        row.id_str,
                        row.tweet_url,
                        row.publish_date,
                        str(row.tweet_user['username']),
                        row.lang,
                        row.rawContent,
                        row.replyCount,
                        row.retweetCount,
                        row.likeCount,
                        row.quoteCount,
                        row.conversationId,
                        row.hashtags,
                        row.cashtags,
                        (list(pd.DataFrame(row.mentionedUsers)['username']) if str(row.mentionedUsers) != '[]' else row.mentionedUsers),
                        str(row.links),
                        row.viewCount,
                        row.retweetedTweet['rawContent'] if (row.retweetedTweet != None) else row.retweetedTweet,
                        str(row.quotedTweet),
                        str(row.place),
                        str(row.coordinates),
                        str(row.inReplyToTweetId),
                        str(row.inReplyToUser),
                        str(row.source),
                        str(row.sourceUrl),
                        str(row.sourceLabel),
                        str(row.media),
                        str(row._type)
                       )
        )
        
    except psycopg2.errors.UniqueViolation:
        print('Tweet already in database. Tweet ID: ', row.id)
    except BaseException as ex:
            ex_type, ex_value, ex_traceback = sys.exc_info()
            
            # Extract unformatter stack traces as tuples
            trace_back = traceback.extract_tb(ex_traceback)

            # Format stacktrace
            stack_trace = list()

            for trace in trace_back:
                stack_trace.append("File : %s , Line : %d, Func.Name : %s, Message : %s" % (trace[0], trace[1], trace[2], trace[3]))
            print('Error at: ', row.id, 'Error: ', ex_value)
            print("Exception type : %s " % ex_type.__name__)
            print("Exception message : %s" %ex_value)
            print("Stack trace : %s" %stack_trace)
            print('______________')

## adjust attribute types for lists to sql array as done for mentionedUsers

Tweet already in database. Tweet ID:  1720010313473405173
Tweet already in database. Tweet ID:  1714613104431247428
Tweet already in database. Tweet ID:  1708796474174152973
Tweet already in database. Tweet ID:  1345753534168506370
Tweet already in database. Tweet ID:  1345539000312991750
Tweet already in database. Tweet ID:  1721873099032023504
Tweet already in database. Tweet ID:  1714914286391226553
Tweet already in database. Tweet ID:  1721496685136777287
Tweet already in database. Tweet ID:  1345205662222057472
